In [80]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import tensorflow as tf

from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/processed_salaries.csv")
df.head()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,work_year,experience_level,employment_type,job_title,salary_in_usd,employee_residence,remote_ratio,company_size,salary_binned
0,2024,EN,CT,Other,88888,Other,0,L,0
1,2024,EX,FT,Other,106666,Other,100,L,0
2,2024,EX,FT,Other,106666,Other,100,L,0
3,2024,SE,FT,Data Engineer,130500,US,0,M,0
4,2024,SE,FT,Data Engineer,96000,US,0,M,0


In [82]:
#Convert categorical data to numeric with 'pd.get_dummies'
df = pd.get_dummies(df, columns=['experience_level',  'job_title'])
df.head()

#removing employee_residence

,work_year,employment_type,salary_in_usd,employee_residence,remote_ratio,company_size,salary_binned,experience_level_EN,experience_level_EX,experience_level_MI,experience_level_SE,job_title_Data Analyst,job_title_Data Engineer,job_title_Data Scientist,job_title_Machine Learning Engineer,job_title_Other
0,2024,CT,88888,Other,0,L,0,1,0,0,0,0,0,0,0,1
1,2024,FT,106666,Other,100,L,0,0,1,0,0,0,0,0,0,1
2,2024,FT,106666,Other,100,L,0,0,1,0,0,0,0,0,0,1
3,2024,FT,130500,US,0,M,0,0,0,0,1,0,1,0,0,0
4,2024,FT,96000,US,0,M,0,0,0,0,1,0,1,0,0,0


In [83]:
# Split our preprocessed data into our features and target arrays. Here we make salary_in_usd as our target array as that is what we would like predicted
y = df['salary_binned'].values
X = df.drop(columns=[ 'salary_in_usd', 'salary_binned', 'employee_residence', 'employment_type', 'company_size', 'work_year']).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [84]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [85]:
# Convert target variable to one-hot encoded format
num_bins = 5
num_classes = num_bins
y_train_encoded = tf.keras.utils.to_categorical(y_train, num_classes=num_classes)

# Compile, Train and Evaluate the Model

In [86]:
#Define the model architecture
model = tf.keras.models.Sequential([
   tf.keras.layers.Dense(128, input_dim=X_train.shape[1], activation=tf.keras.activations.gelu),
   tf.keras.layers.Dense(64, activation=tf.keras.activations.gelu),
   tf.keras.layers.Dense(32, activation=tf.keras.activations.gelu),
   tf.keras.layers.Dense(num_bins, activation='softmax')
])

#Check the structure of the model
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_23 (Dense)            (None, 128)               1408      
                                                                 
 dense_24 (Dense)            (None, 64)                8256      
                                                                 
 dense_25 (Dense)            (None, 32)                2080      
                                                                 
 dense_26 (Dense)            (None, 5)                 165       
                                                                 
Total params: 11909 (46.52 KB)
Trainable params: 11909 (46.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [87]:
# Compile the model with appropriate loss function for multi-class classification
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [88]:
# Train the model
model.fit(X_train_scaled, y_train_encoded, epochs=100, verbose=1)

Epoch 1/100
267/267 [==============================] - 2s 3ms/step - loss: 1.0199 - accuracy: 0.5750
Epoch 2/100
267/267 [==============================] - 1s 3ms/step - loss: 0.9402 - accuracy: 0.5838
Epoch 3/100
267/267 [==============================] - 1s 2ms/step - loss: 0.9355 - accuracy: 0.5845
Epoch 4/100
267/267 [==============================] - 1s 2ms/step - loss: 0.9334 - accuracy: 0.5852
Epoch 5/100
267/267 [==============================] - 1s 3ms/step - loss: 0.9346 - accuracy: 0.5845
Epoch 6/100
267/267 [==============================] - 1s 3ms/step - loss: 0.9346 - accuracy: 0.5856
Epoch 7/100
267/267 [==============================] - 1s 3ms/step - loss: 0.9328 - accuracy: 0.5850
Epoch 8/100
267/267 [==============================] - 1s 3ms/step - loss: 0.9334 - accuracy: 0.5844
Epoch 9/100
267/267 [==============================] - 1s 3ms/step - loss: 0.9327 - accuracy: 0.5850
Epoch 10/100
267/267 [==============================] - 1s 3ms/step - loss: 0.9330 - accura

In [89]:
# Evaluate the model on the test set
model_loss, model_accuracy = model.evaluate(X_test_scaled, tf.keras.utils.to_categorical(y_test, num_classes=num_classes), verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

89/89 - 0s - loss: 0.9571 - accuracy: 0.5673 - 298ms/epoch - 3ms/step
Loss: 0.9570911526679993, Accuracy: 0.5673110485076904
